In [82]:
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import random

In [83]:
#Things to record-
#(1) Distribution of area of toppling affected by one sanddrop. 
#(2) Dsitribution of time of toppling due to one sanddrop.  
#(3) Distribution of number of topplings on adding one sanddrop. (one time step can have >1 unstable points)

In [97]:
def generate_grid(dim, z_c):              
   #Create a grid for operations
    grid_1d=np.zeros((dim + 2)*(dim + 2)*2, dtype=int)
    grid=np.reshape(grid_1d,(dim+2,dim+2,2))  # 3rd dim for marking affected blocks.
    
    # random initialisation of the grid with boundaries fixed at 0
    for i in range(1, dim+1):
        for j in range(1,dim+1):
            grid[i,j,0]=int(np.random.random()*z_c)
    grid[:,0,0]=0
    grid[0,:,0]=0
    grid[:,dim+1,0]=0
    grid[dim+1,:,0]=0
    
    print(grid[:,:,0])
    return grid


def dropsand(grid_old, dim):    # add a grain of sand to the grid
    grid=grid_old.copy()
    x=random.randint(1,dim+1)
    y=random.randint(1,dim+1)
    grid[x,y,0]=grid[x,y,0]+1
    return grid
       

def topplefunc(grid_recieved, dim, unstable_sites):     # to topple the sites after noting down the unstable ones.
    grid=grid_recieved.copy()
    # resetting the affected sites
    for x in unstable_sites:
        grid[x[0],x[1],0]=grid[x[0],x[1],0]-4
        grid[x[0],x[1]+1,0]=grid[x[0],x[1]+1,0]+1
        grid[x[0]+1,x[1],0]=grid[x[0]+1,x[1],0]+1
        grid[x[0],x[1]-1,0]=grid[x[0],x[1]-1,0]+1
        grid[x[0]-1,x[1],0]=grid[x[0]-1,x[1],0]+1
        grid[x[0],x[1],1]=1
        grid[x[0],x[1]+1,1]=1
        grid[x[0]+1,x[1],1]=1
        grid[x[0],x[1]-1,1]=1
        grid[x[0]-1,x[1],1]=1
        
    # resetting boundaries to 0
    grid[:,0,0]=0
    grid[0,:,0]=0
    grid[:,dim+1,0]=0
    grid[dim+1,:,0]=0
    grid[:,0,1]=0
    grid[0,:,1]=0
    grid[:,dim+1,1]=0
    grid[dim+1,:,1]=0
    
    # return the updated grid which has been toppled and has affected areas marked.
    return grid.copy()
    

def count_area(grid, time):    # returning the number of blocks affected
    counter=0
    for i in range(1, dim+1):
        for j in range(1, dim+1):
            if(grid[i,j,1]==1):
                counter=counter+1
    return counter


def count_topple(list):       # return the sum of elements of the "no_of_unstable_sites" list.
    sum=0
    for n in list:
        sum=sum+n
    return sum

def count_time(list):        # the number of elements appended to "no_of_unstable_sites" is a direct measure of relax time.
    time=len(list)-1         # we subtract 1 because the while loop runs an extra iteration
    return time
    
def updategrid(grid_old, dim, z_c):
    grid=grid_old.copy()
    grid[:,:,1]=0                  # resetting the affected sites to null
    unstable=1                     # initially, we set unstability factor to 1 implying True.
    no_of_unstable_sites=[]
    
    while(unstable!=0):            # while unstability doesn't turn to False.
        unstable_sites=[]
        for i in range(1,dim+1):
            for j in range(1,dim+1):
                if(grid[i,j,0]>=z_c):
                    unstable_sites.append([i,j])     # noting down the unstable points at time step t
        
        unstable=len(unstable_sites)                 # as soon as there are 0 unstable points, we exit the while loop.
        no_of_unstable_sites.append(unstable)        # appending number of unstable sites. sum them to get total topplings.
        
        grid=topplefunc(grid, dim, unstable_sites)
    # ------while loop ends-----
    
    area=count_area(grid, dim)
    topples=count_topple(no_of_unstable_sites)
    relaxtime=count_time(no_of_unstable_sites)
    
    return grid.copy(),area,topples,relaxtime
    

In [98]:
dim=25
z_c=4
time_steps=20
area_list=[]
topples_list=[]
relaxtime_list=[]
grid=generate_grid(dim, z_c)
start = timer()

# evolving the grid in time t
for t in range(time_steps):
    grid_sanddrop= dropsand(grid, dim)
    grid,area,topples,relaxtime=updategrid(grid_sanddrop, dim, z_c)
    
    area_list.append(area)
    topples_list.append(topples)
    relaxtime_list.append(relaxtime)
    
end = timer()
print("Runtime= ", end-start)
print("Area= ", area_list)
print("Topples= ",topples_list)
print("Relaxation Time= ", relaxtime_list) 

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 3 3 2 2 0 1 3 1 3 2 3 3 0 1 2 0 3 3 2 3 2 3 0 0 0]
 [0 2 1 3 3 0 2 1 2 2 1 2 3 0 2 0 1 1 3 1 3 0 2 0 0 3 0]
 [0 3 1 3 1 1 1 1 3 3 3 3 1 0 2 1 1 3 2 3 3 2 0 1 2 1 0]
 [0 2 3 0 2 3 0 1 1 2 0 3 1 1 0 2 1 2 1 2 2 1 3 0 0 0 0]
 [0 1 1 1 3 0 0 3 1 1 1 2 0 1 2 0 2 1 1 0 0 3 2 2 1 0 0]
 [0 1 0 2 0 2 3 0 3 3 1 2 0 0 2 3 0 3 1 2 0 0 2 3 2 2 0]
 [0 2 0 2 3 0 2 2 1 1 1 1 1 0 0 2 3 2 3 1 3 1 3 2 1 1 0]
 [0 0 1 2 3 3 3 2 2 2 1 0 0 2 3 0 2 1 2 2 2 0 2 0 0 0 0]
 [0 1 1 2 3 0 2 0 3 3 2 3 1 3 1 1 1 1 1 2 3 3 3 3 2 1 0]
 [0 3 2 2 2 0 2 3 0 2 1 3 3 2 2 2 1 2 3 0 2 3 2 1 1 2 0]
 [0 3 2 2 0 0 3 2 3 3 0 0 3 0 0 1 1 0 3 0 2 0 2 1 2 3 0]
 [0 2 1 3 2 0 2 2 2 0 3 2 1 3 3 3 1 1 1 0 3 1 0 3 1 1 0]
 [0 1 2 3 1 0 1 0 3 3 0 2 3 1 1 1 0 2 1 2 3 3 2 1 3 3 0]
 [0 0 3 1 3 2 3 1 2 2 1 0 0 2 0 0 3 2 0 1 2 3 3 2 3 3 0]
 [0 3 3 1 3 2 3 3 2 3 0 1 3 2 0 0 1 2 1 1 2 2 3 0 0 2 0]
 [0 0 3 2 3 0 2 2 3 2 0 0 3 1 0 3 0 3 0 1 1 2 2 0 0 3 0]
 [0 0 1 0 0 1 2 1 3 1 0 2 2 2 2